<a href="https://colab.research.google.com/github/rahiakela/practical-machine-learning-with-tensorflow/blob/week-4/assignment_4_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
!pip install tensorflow==2.0.0-rc0

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
import tensorflow as tf
import pandas as pd

tf.random.set_seed(1)

In [0]:
boston_dataset = load_boston()

data_X = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
data_Y = pd.DataFrame(boston_dataset.target, columns=["target"])
data = pd.concat([data_X, data_Y], axis=1)

In [27]:
train, test = train_test_split(data, test_size=0.2, random_state=1)
train, val = train_test_split(train, test_size=0.2, random_state=1)
print(len(train), "train examples")
print(len(val), "validation examples")
print(len(test), "test examples")

323 train examples
81 validation examples
102 test examples


In [28]:
# analyse dataset
data_X.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [29]:
data_Y.head()

,target
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [0]:
# Converting the Pandas DataFrames into Tensorflow Datasets
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 32
train_ds = df_to_dataset(train, True, batch_size)
val_ds = df_to_dataset(val, False, batch_size)
test_ds = df_to_dataset(test, False, batch_size)  

7) Define all the feature columns to be numerical columns, build and compile the model based on the instructions in the notebook and train
	   the model for 200 epochs. What is the range of the training error at the end of the training process?

In [43]:
from tensorflow import feature_column
# Defining Feature Columns
CRIM = feature_column.numeric_column('CRIM')
ZN = feature_column.numeric_column('ZN')
INDUS = feature_column.numeric_column('INDUS')
CHAS = feature_column.numeric_column('CHAS')
NOX = feature_column.numeric_column('NOX')
RM = feature_column.numeric_column('RM')
AGE = feature_column.numeric_column('AGE')
DIS = feature_column.numeric_column('DIS')
RAD = feature_column.numeric_column('RAD')
TAX = feature_column.numeric_column('TAX')
PTRATIO = feature_column.numeric_column('PTRATIO')
B = feature_column.numeric_column('B')
LSTAT = feature_column.numeric_column('LSTAT')
# define feature_columns as a list of features using functions from tf.feature_column
feature_columns = []
for header in ['CRIM', 'ZN', 'INDUS', 'CHAS',	'NOX', 'RM', 'AGE', 'DIS', 'RAD',	'TAX', 'PTRATIO',	'B', 'LSTAT']:
  feature_columns.append(feature_column.numeric_column(header))
#for header in [CRIM,	ZN,	INDUS,	CHAS,	NOX,	RM,	AGE,	DIS,	RAD,	TAX,	PTRATIO,	B,	LSTAT]:
#  feature_columns.append(feature_column.numeric_column(header))
feature_columns[:5]  

[NumericColumn(key='CRIM', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='ZN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='INDUS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='CHAS', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NOX', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [0]:
# Building the model
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
model = tf.keras.Sequential([
   feature_layer,
   tf.keras.layers.Dense(1, activation=None)                         
])

# compile model
model.compile(optimizer='adam', loss='mse', metrcis=['mse'])

In [45]:
model.fit(train_ds, validation_data=val_ds, epochs=200)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/200
11/11 [==============================] - 1s 64ms/step - loss: 136907.7827 - val_loss: 0.0000e+00
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 126439.5245 - val_loss: 113610.8333
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 119202.4500 - val_loss: 107338.4167
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 112764.0309 - val_loss: 101270.8099
Epoch 5/200
11/11 [==============================] - 0s 4ms/step - loss:

8) Change the model architecture by a hidden layer of 4 units and use sigmoid activation. Train the model for 200 epochs. What is the
	   range of the training error at the end of the training process? Try to think of why this is happening.

In [35]:
#loss, mse = model.evaluate(test_ds)
#print("Mean Squared Error - Test Data", mse)

model = tf.keras.Sequential([
   feature_layer,
   tf.keras.layers.Dense(4, activation='sigmoid')                         
])

# compile model
model.compile(optimizer='adam', loss='mse', metrcis=['mse'])
model.fit(train_ds, validation_data=val_ds, epochs=200)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/200
11/11 [==============================] - 1s 62ms/step - loss: 597.7718 - val_loss: 0.0000e+00
Epoch 2/200
11/11 [==============================] - 0s 4ms/step - loss: 580.8497 - val_loss: 546.9360
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 576.3347 - val_loss: 546.9147
Epoch 4/200
11/11 [==============================] - 0s 4ms/step - loss: 579.0013 - val_loss: 546.8695
Epoch 5/200
11/11 [==============================] - 0s 4ms/step - loss: 578.4749 - val_loss:

9) We will now experiment by changing the loss function. First, remove the hidden layer that we added for Q8. Then, try the solutions
	   of the Q6 as the loss function for the optimizer. What is the mean squared error on the test dataset for the best model? (approx)

In [38]:
model_3 = tf.keras.Sequential([
   feature_layer,
   tf.keras.layers.Dense(1, activation=None)                         
])

# compile model
model_3.compile(optimizer='adam', loss='mse', metrcis=['mse', 'mae'])
model_3.fit(train_ds, validation_data=val_ds, epochs=200)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/200
11/11 [==============================] - 1s 81ms/step - loss: 29777.7112 - val_loss: 0.0000e+00
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 27764.9200 - val_loss: 22820.7819
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 24522.5009 - val_loss: 20157.6302
Epoch 4/200
11/11 [==============================] - 0s 5ms/step - loss: 21722.8178 - val_loss: 17691.7539
Epoch 5/200
11/11 [==============================] - 0s 4ms/step - loss: 19043.

10) We will now try to bucketize one of the feature columns and see its effect on the model’s performance. Bucketize the ‘RAD’ column
	   with the ​ boundaries ​ parameter as [2, 5] and retrain the model for 200 epochs. The performance on the test data:

In [46]:
rad = feature_column.numeric_column('RAD')
rad_buckets = feature_column.bucketized_column(rad, boundaries=[2, 5])
feature_columns.append(rad_buckets)

# Building the model
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
model_4 = tf.keras.Sequential([
   feature_layer,
   tf.keras.layers.Dense(1, activation=None)                         
])

# compile model
model_4.compile(optimizer='adam', loss='mse', metrcis=['mse'])
model_4.fit(train_ds, validation_data=val_ds, epochs=200)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1/200
11/11 [==============================] - 1s 88ms/step - loss: 17003.0692 - val_loss: 0.0000e+00
Epoch 2/200
11/11 [==============================] - 0s 5ms/step - loss: 15094.3410 - val_loss: 14485.7578
Epoch 3/200
11/11 [==============================] - 0s 4ms/step - loss: 13136.0439 - val_loss: 12664.8984
Epoch 4/200
11/11 [==============================] - 0s 5ms/step - loss: 11283.4205 - val_loss: 11098.3031
Epoch 5/200
11/11 [==============================] - 0s 5ms/step - loss: 9812.8